# Things to do
- EDA: inspect columns, handle missing data, define target
- Build baseline Logistic Regression model
- Add Decision Tree + Random Forest, compare scores
- Model selection + tuning (max_depth, n_estimators, etc.)
- Train final model & save with pickle
- Implement FastAPI service for inference
- Dockerize the app (Dockerfile)
- Test locally + via curl or Postman
- Optional cloud deploy (Render / Railway / Fly.io)

Questions:
- what does imbalanced data mean?
- Show a bar chart about the imbalance data

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import export_text

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score, accuracy_score

### Load saved splits and target variables

In [6]:
# Load data splits
df_full_train = pd.read_parquet("../data/interim/df_full_train.parquet")
df_train = pd.read_parquet("../data/interim/df_train.parquet")
df_val = pd.read_parquet("../data/interim/df_val.parquet")
df_test = pd.read_parquet("../data/interim/df_test.parquet")

# Load target variables
y_train = np.load("../data/interim/y_train.npy")
y_val = np.load("../data/interim/y_val.npy")
y_test = np.load("../data/interim/y_test.npy")

In [7]:
numerical = [
    'time_in_hospital',
    'num_lab_procedures',
    'num_procedures',
    'num_medications',
    'number_outpatient',
    'number_emergency',
    'number_inpatient',
    'number_diagnoses'
]

In [8]:
cat_cols = ['race', 'gender', 'age', 'diag_1', 'diag_2', 'diag_3', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed']

# Feature Importance: 
- Readmission Rate
- Risk ratio
- Mutual information
- Correlation


### Readmission Rate
#### Best for Binary variables
Questions:
- Is feature importance only done on categorical variables?
- How does the mean rate tell us the important features?
- Should we do it on all features

In [9]:
global_readmitted = df_full_train.readmitted.mean()
global_readmitted

np.float64(0.11261239129366678)

In [10]:
readmitted_female = df_full_train[df_full_train.gender == 'female'].readmitted.mean()
readmitted_female

np.float64(0.11311752351383435)

In [22]:
readmitted_male = df_full_train[df_full_train.gender == 'male'].readmitted.mean()
readmitted_male

np.float64(0.11203297433851882)

In [23]:
global_readmitted - readmitted_female

np.float64(-0.0005051322201675701)

In [24]:
global_readmitted - readmitted_male

np.float64(0.00057941695514796)

### Risk Ratio
### Best for Categorical groups
Question: 
- what does the risk and diff tell us about these features

In [25]:
readmitted_female / global_readmitted

np.float64(1.0044855829306591)

In [26]:
readmitted_male / global_readmitted

np.float64(0.9948547673262974)

In [27]:
from IPython.display import display

In [28]:
#for c in cat_cols:
    # print(c)
    # df_group = df_full_train.groupby(c).readmitted.agg(['mean', 'count'])
    # df_group['diff'] = df_group['mean'] - global_readmitted
    # df_group['risk'] = df_group['mean'] / global_readmitted
    # display(df_group)
    # print()
    # print()

### Mutual information
### Best for Categorical groups
How much info a feature gives about target
- ✅ MI close to 0 → no info
- ✅ Higher MI → strong informative feature

In [29]:
from sklearn.metrics import mutual_info_score

In [30]:
def mutual_info_readmitted_score(series):
    return mutual_info_score(series, df_full_train.readmitted)

In [31]:
mi = df_full_train[cat_cols].apply(mutual_info_readmitted_score)
mi.sort_values(ascending=False)

diag_1                      8.969244e-03
diag_3                      8.143188e-03
diag_2                      7.503651e-03
insulin                     9.084641e-04
age                         5.814475e-04
diabetesMed                 4.114289e-04
metformin                   2.986104e-04
change                      1.700384e-04
race                        1.198009e-04
glipizide                   5.710555e-05
acarbose                    5.256147e-05
pioglitazone                4.801782e-05
repaglinide                 4.431814e-05
rosiglitazone               3.187767e-05
chlorpropamide              3.154034e-05
miglitol                    2.537009e-05
glyburide                   1.840014e-05
glimepiride                 1.602352e-05
glyburide-metformin         1.571878e-05
nateglinide                 1.537855e-05
gender                      5.865645e-06
tolbutamide                 4.334145e-06
metformin-rosiglitazone     2.935070e-06
troglitazone                2.935070e-06
tolazamide      

### Correlation
### Best for numerical features
- ➡️ High positive correlation → increases chance of readmission
- ➡️ High negative correlation → decreases it

In [23]:
df_full_train[numerical].corrwith(df_full_train.readmitted).abs()

time_in_hospital      0.044728
num_lab_procedures    0.019398
num_procedures        0.012659
num_medications       0.037057
number_outpatient     0.016453
number_emergency      0.059897
number_inpatient      0.166797
number_diagnoses      0.048522
dtype: float64

# One-hot encoding

In [20]:
dv = DictVectorizer(sparse=False)

In [21]:
train_dict = df_train[list(cat_cols) + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[list(cat_cols) + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

# Training Logistic Regression

In [22]:
model = LogisticRegression(solver='lbfgs')

model.fit(X_train, y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [37]:
y_pred = model.predict_proba(X_val)[:, 1]

In [38]:
readmitted_decision = (y_pred >= 0.5)

In [39]:
# Accuracy
(y_val == readmitted_decision).mean()

np.float64(0.887633272736206)

In [40]:
# Coefficients
# Positive coefficient (+) ⇒ increases the chance of being readmitted

# Negative coefficient (−) ⇒ decreases the chance of readmission

feature_names = dv.get_feature_names_out()
coefs = model.coef_[0]
importance = sorted(zip(feature_names, coefs), key=lambda x: abs(x[1]), reverse=True)
importance[:10]

[('number_inpatient', np.float64(0.24554441753692352)),
 ('diabetesMed=no', np.float64(-0.1951111052170394)),
 ('diag_1=434', np.float64(0.18026873768211224)),
 ('change=ch', np.float64(-0.17921927235217283)),
 ('insulin=no', np.float64(-0.1638967091246426)),
 ('diag_2=250', np.float64(-0.156737405564199)),
 ('diag_3=401', np.float64(-0.15477725217671787)),
 ('nateglinide=no', np.float64(-0.153098836474299)),
 ('metformin=steady', np.float64(-0.15137360540453587)),
 ('diag_1=786', np.float64(-0.14735120787950357))]

## Evaluation of Logistic Regression

### Accuracy

In [41]:
C_grid = [0.01, 0.1, 1, 10, 100]
accuracies = []

for C in C_grid:

    # Create the model instance
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=1)
    model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    print(C)
    converted_decision = (y_pred >= 0.5)
    accuracy = (y_val == converted_decision).mean()
    accuracies.append({C: float(round(accuracy, 3))})

print(accuracies)

0.01
0.1
1
10
100
[{0.01: 0.888}, {0.1: 0.888}, {1: 0.888}, {10: 0.888}, {100: 0.888}]


### Confusion Matrix

In [42]:

cm = confusion_matrix(y_val, readmitted_decision)
cm


array([[18031,    31],
       [ 2256,    35]])

### Precision and Recall

In [43]:
precision = precision_score(y_val, readmitted_decision)

recall = recall_score(y_val, readmitted_decision)

print(f"Precision: {precision: .3f}")
print(f"Recall: {recall: .3f}")

Precision:  0.530
Recall:  0.015


### ROC Curve & AUC Score¨
0.5 = random guessing

1.0 = perfect model

0.8 = strong model

In [44]:
auc = roc_auc_score(y_val, y_pred)

print(f"AUC: {auc: .3f}")

AUC:  0.632


In [45]:
C_grid = [0.01, 0.1, 1, 10, 100]
scores = []

for C in C_grid:

    # Create the model instance
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=1)
    model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    print(C)
    converted_decision = (y_pred >= 0.5)
    accuracy = (y_val == converted_decision).mean()
    
    precision = precision_score(y_val, readmitted_decision)
    
    recall = recall_score(y_val, readmitted_decision)

    auc = roc_auc_score(y_val, y_pred)
    scores.append({
        'C': C,
        'accuracy': round(accuracy, 3),
        'precision': round(precision, 3),
        'recall': round(recall, 3),
        'auc': round(auc, 3)
    })

print(scores)

0.01
0.1
1
10
100
[{'C': 0.01, 'accuracy': np.float64(0.888), 'precision': 0.53, 'recall': 0.015, 'auc': 0.632}, {'C': 0.1, 'accuracy': np.float64(0.888), 'precision': 0.53, 'recall': 0.015, 'auc': 0.632}, {'C': 1, 'accuracy': np.float64(0.888), 'precision': 0.53, 'recall': 0.015, 'auc': 0.632}, {'C': 10, 'accuracy': np.float64(0.888), 'precision': 0.53, 'recall': 0.015, 'auc': 0.632}, {'C': 100, 'accuracy': np.float64(0.888), 'precision': 0.53, 'recall': 0.015, 'auc': 0.632}]


### Thresholds, plotting of graph, K-fold - come back to this